In [1]:
import pandas as pd
import requests
import json
from datetime import date

## Get county raw data

### County Map
The County of Santa Clara has published this ArcGIS [map](https://csj.maps.arcgis.com/apps/webappviewer/index.html?appid=ddf8ba621416421ebd3fa5ebeeb5e684) of free meal food programs during COVID-19. 

#### How did we find this link?

This map was found by Charlie/Chris LeBoa/Joyce (I'm not sure how) and placed on the data wishlist.

### Identifying the Datasets
The map shows various layers of data, which are different datasets we may want to pull. To view these layers, open the map's Turn On/Off Layers menu (top right corner, the button that looks like three overlapping diamonds). 

Click on the three dots next to one of the layers. Click on "Description" in the dropdown menu that appears. It should lead you to a link like this: https://geo.sanjoseca.gov/server/rest/services/EOC/EOC_COVID19_SantaClaraCountyFoodAccess/MapServer/1 This page appears to contain the backend data that is displayed on the frontend map we just came from. Some relevant parts of the page:

- **At the top of the page:** The top of the page should look like this:
```
Home > services > EOC > EOC_COVID19_SantaClaraCountyFoodAccess (MapServer) > Food Senior Meal Sites
```
  - These links show us the tree structure of all datasets that the Santa Clara County ArcGIS account appears to manage. We are currently on the Food Senior Meal Sites dataset. 
  - If we click on [EOC_COVID19_SantaClaraCountyFoodAccess (MapServer)](https://geo.sanjoseca.gov/server/rest/services/EOC/EOC_COVID19_SantaClaraCountyFoodAccess/MapServer), we actually see a list of 7 datasets (layers) for the map. Note that some of these layers are hidden from the frontend map, which is why this is not a one-to-one correspondence with the list you saw on the map's Layers menu.
  - The cell below pulls all 7 datasets by looping through the IDs (1-7).
- **In the page:** Shows us metadata about this dataset and a list of its fields (near the bottom of the page)
- **At the bottom of the page:** Note the [query](https://geo.sanjoseca.gov/server/rest/services/EOC/EOC_COVID19_SantaClaraCountyFoodAccess/MapServer/1/query) link. This gives us a web interface to query for the data in this dataset
  - In our scraping script, we will recreate the web request that is sent when you use this web interface. See the cell below for the mapping from HTTP request parameters to the web interface's form fields.

In [2]:
# Query for all data from each dataset, store in an array of Pandas dataframes
url = "https://services.arcgis.com/6kSayNlqm3HvsYZ8/ArcGIS/rest/services/EOC_COVID19_SantaClaraCountyFoodAccess/FeatureServer/%s/query"
dfs = []

params = {
    "f":"json",              # Corresponds to "Format" in the web interface
    "where":"1=1",           # Corresponds to "Where"; 1=1 means get all the data
    "returnGeometry":"true", # Corresponds to "Return Geometry"
    "outFields":"*",         # Corresponds to "Out Fields"; * means get all fields
}

for dataset_id in range(1, 8):
  res = requests.get(url % dataset_id, params=params)
  df = pd.json_normalize(json.loads(res.text)['features'])
  dfs.append(df)

In [6]:
# Download each dataset as an individual CSV
today = date.today()

for i, df in enumerate(dfs):
    filename = 'santa_clara_raw_%d_%s.csv' % (i, today.strftime("%m_%d_%Y"))
    df.to_csv(filename)
    

Check the folder on your computer where this Python notebook is stored. You should see 7 new CSVs!